# Módulo 3 - Modelado

## Objetivos

## Librerías

In [1]:
# Importamos la función para dividir datos en entrenamiento y prueba
from sklearn.model_selection import train_test_split as sklearn_train_test_split

# Importamos la función para calcular similitud coseno entre vectores
from sklearn.metrics.pairwise import cosine_similarity

# Importamos dos tipos de vectorizadores de texto:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# Importamos librerías estándar para trabajo numérico y análisis de datos
import numpy as np
import pandas as pd

# Importamos kagglehub, probablemente para cargar modelos o datasets desde la plataforma de Kaggle
import kagglehub


/home/druiz35/Documents/SEpsilon/SEpsilon-Proyectos/AI-ML-Data/RNABI2025-1-Equipo3/Trabajo3/.surprise_venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Cargado de datos

In [2]:
# Download latest version
general_path = kagglehub.dataset_download("amanmehra23/travel-recommendation-dataset")

print("Path to dataset files:", general_path)

Path to dataset files: /home/druiz35/.cache/kagglehub/datasets/amanmehra23/travel-recommendation-dataset/versions/1


In [3]:
destinations_path = general_path + "/Expanded_Destinations.csv"
destinations_df = pd.read_csv(destinations_path)

reviews_path = general_path + "/Final_Updated_Expanded_Reviews.csv"
reviews_df = pd.read_csv(reviews_path)

userhistory_path = general_path + "/Final_Updated_Expanded_UserHistory.csv"
userhistory_df = pd.read_csv(userhistory_path)

users_path = general_path + "/Final_Updated_Expanded_Users.csv"
users_df = pd.read_csv(users_path)

dataframes = {
    "Destinations": destinations_df,
    "Reviews": reviews_df,
    "User History": userhistory_df,
    "Users":users_df
}

In [4]:
# Definimos la ruta del archivo CSV que contiene el DataFrame final previamente guardado
path = "m3_merged_df.csv"

# Leemos el archivo CSV y lo cargamos en un DataFrame llamado 'df'
df = pd.read_csv(path)

# Configuramos pandas para que muestre todas las columnas del DataFrame al imprimirlo
# Esto es útil cuando el DataFrame tiene muchas columnas que normalmente se truncarían en la vista
pd.set_option('display.max_columns', None)

# Mostramos el DataFrame cargado
df


,Unnamed: 0,ReviewID,DestinationID_x,UserID,Rating,ReviewText,Name_x,State,Type,Popularity,BestTimeToVisit,HistoryID,DestinationID_y,VisitDate,ExperienceRating,Name_y,Email,Preferences,Gender,NumberOfAdults,NumberOfChildren
0,0,1,178,327,2,Incredible monument!,Jaipur City,Rajasthan,City,8.544352,Oct-Mar,79,175,2024-01-01,3,Pooja,pooja@example.com,"City, Historical",Female,1,1
1,1,2,411,783,1,Loved the beaches!,Taj Mahal,Uttar Pradesh,Historical,8.284127,Nov-Feb,834,894,2024-03-20,2,Karan,karan@example.com,"City, Historical",Male,1,1
2,2,4,358,959,3,Incredible monument!,Jaipur City,Rajasthan,City,7.738761,Oct-Mar,998,660,2024-02-15,4,Ritvik,ritvik@example.com,"Nature, Adventure",Male,1,1
3,3,5,989,353,2,Loved the beaches!,Kerala Backwaters,Kerala,Nature,8.208088,Sep-Mar,202,894,2024-01-01,5,Isha,isha@example.com,"Nature, Adventure",Female,2,0
4,4,6,473,408,4,A historical wonder,Jaipur City,Rajasthan,City,8.138558,Oct-Mar,331,403,2024-01-01,2,Ishaan,ishaan@example.com,"City, Historical",Male,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,988,991,701,850,3,Incredible monument!,Taj Mahal,Uttar Pradesh,Historical,8.814029,Nov-Feb,138,131,2024-03-20,1,Hitesh,hitesh@example.com,"Beaches, Historical",Male,2,0
989,989,991,701,850,3,Incredible monument!,Taj Mahal,Uttar Pradesh,Historical,8.814029,Nov-Feb,643,761,2024-01-01,4,Hitesh,hitesh@example.com,"Beaches, Historical",Male,2,0
990,990,995,231,346,5,Loved the beaches!,Taj Mahal,Uttar Pradesh,Historical,7.788256,Nov-Feb,454,113,2024-01-01,2,Hitesh,hitesh@example.com,"Beaches, Historical",Male,2,2
991,991,995,231,346,5,Loved the beaches!,Taj Mahal,Uttar Pradesh,Historical,7.788256,Nov-Feb,556,128,2024-01-01,4,Hitesh,hitesh@example.com,"Beaches, Historical",Male,2,2


In [5]:
# Se dropea la columna de índices sin nombre ya que no contribuirá de ninguna forma al análisis
df = df.drop(df.columns[[0,1]], axis=1)
df

,DestinationID_x,UserID,Rating,ReviewText,Name_x,State,Type,Popularity,BestTimeToVisit,HistoryID,DestinationID_y,VisitDate,ExperienceRating,Name_y,Email,Preferences,Gender,NumberOfAdults,NumberOfChildren
0,178,327,2,Incredible monument!,Jaipur City,Rajasthan,City,8.544352,Oct-Mar,79,175,2024-01-01,3,Pooja,pooja@example.com,"City, Historical",Female,1,1
1,411,783,1,Loved the beaches!,Taj Mahal,Uttar Pradesh,Historical,8.284127,Nov-Feb,834,894,2024-03-20,2,Karan,karan@example.com,"City, Historical",Male,1,1
2,358,959,3,Incredible monument!,Jaipur City,Rajasthan,City,7.738761,Oct-Mar,998,660,2024-02-15,4,Ritvik,ritvik@example.com,"Nature, Adventure",Male,1,1
3,989,353,2,Loved the beaches!,Kerala Backwaters,Kerala,Nature,8.208088,Sep-Mar,202,894,2024-01-01,5,Isha,isha@example.com,"Nature, Adventure",Female,2,0
4,473,408,4,A historical wonder,Jaipur City,Rajasthan,City,8.138558,Oct-Mar,331,403,2024-01-01,2,Ishaan,ishaan@example.com,"City, Historical",Male,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,701,850,3,Incredible monument!,Taj Mahal,Uttar Pradesh,Historical,8.814029,Nov-Feb,138,131,2024-03-20,1,Hitesh,hitesh@example.com,"Beaches, Historical",Male,2,0
989,701,850,3,Incredible monument!,Taj Mahal,Uttar Pradesh,Historical,8.814029,Nov-Feb,643,761,2024-01-01,4,Hitesh,hitesh@example.com,"Beaches, Historical",Male,2,0
990,231,346,5,Loved the beaches!,Taj Mahal,Uttar Pradesh,Historical,7.788256,Nov-Feb,454,113,2024-01-01,2,Hitesh,hitesh@example.com,"Beaches, Historical",Male,2,2
991,231,346,5,Loved the beaches!,Taj Mahal,Uttar Pradesh,Historical,7.788256,Nov-Feb,556,128,2024-01-01,4,Hitesh,hitesh@example.com,"Beaches, Historical",Male,2,2


## Similarity Score Filtering

In [6]:
df["description"] = (
    df['Type'] + ' ' +
    df['State'] + ' ' +
    df['BestTimeToVisit'] + ' ' +
    df['Preferences'] + ' ' +
    df['Gender'].astype(str) + ' ' +
    df['NumberOfAdults'].astype(str) + ' ' +
    df['NumberOfChildren'].astype(str)
)


In [7]:
# Creamos una instancia de TfidfVectorizer que transforma texto en vectores ponderados por TF-IDF
# Esto reduce el peso de palabras comunes y aumenta el de palabras específicas para cada destino
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['description'])  # Matriz TF-IDF de descripciones


# Creamos una instancia de CountVectorizer que transforma texto en una matriz de conteo de palabras
# Cada fila representa un destino y cada columna la frecuencia de una palabra específica
vectorizer = CountVectorizer(stop_words='english')
destination_features = vectorizer.fit_transform(df['description'])  # Matriz de conteo


In [8]:
# Calculamos la matriz de similitud coseno entre todos los destinos,
# comparando sus vectores de características basados en la descripción
cosine_sim = cosine_similarity(destination_features, destination_features)


In [9]:
cosine_sim

array([[1.        , 0.47140452, 0.72168784, ..., 0.23570226, 0.23570226,
        1.        ],
       [0.47140452, 1.        , 0.13608276, ..., 0.88888889, 0.88888889,
        0.47140452],
       [0.72168784, 0.13608276, 1.        , ..., 0.        , 0.        ,
        0.72168784],
       ...,
       [0.23570226, 0.88888889, 0.        , ..., 1.        , 1.        ,
        0.23570226],
       [0.23570226, 0.88888889, 0.        , ..., 1.        , 1.        ,
        0.23570226],
       [1.        , 0.47140452, 0.72168784, ..., 0.23570226, 0.23570226,
        1.        ]])

In [10]:

def recommend_destinations(user_id, userhistory_df, destinations_df, cosine_sim):
    # Obtenemos los IDs de destinos que el usuario ya visitó o calificó
    visited_destinations = userhistory_df[userhistory_df['UserID'] == user_id]['DestinationID'].values

    # Sumamos las filas de la matriz de similitud correspondientes a esos destinos visitados,
    # creando un vector con la suma total de similitudes a cada destino en el dataset
    similar_scores = np.sum(cosine_sim[visited_destinations - 1], axis=0)

    # Ordenamos los índices de destinos por su puntaje de similitud, de mayor a menor
    recommended_destinations_idx = np.argsort(similar_scores)[::-1]

    recommendations = []
    for idx in recommended_destinations_idx:
        # Excluimos destinos que el usuario ya visitó
        if destinations_df.iloc[idx]['DestinationID'] not in visited_destinations:
            # Añadimos la información relevante del destino recomendado
            recommendations.append(destinations_df.iloc[idx][[
                'DestinationID', 'Name', 'State', 'Type', 'Popularity', 'BestTimeToVisit', 
            ]].to_dict())
        # Limitar a las 5 mejores recomendaciones
        if len(recommendations) >= 5:
            break

    # Retornamos un DataFrame con las recomendaciones finales
    return pd.DataFrame(recommendations)


In [11]:
# Obtenemos recomendaciones para el usuario con ID 2
recommended_destinations = recommend_destinations(
    user_id=2,                   # ID del usuario objetivo
    userhistory_df=userhistory_df, # Historial de visitas/calificaciones del usuario
    destinations_df=destinations_df, # Información de destinos
    cosine_sim=cosine_sim          # Matriz de similitud coseno entre destinos
)

# Mostramos las recomendaciones generadas
recommended_destinations


,DestinationID,Name,State,Type,Popularity,BestTimeToVisit
0,595,Leh Ladakh,Jammu and Kashmir,Adventure,9.446618,Apr-Jun
1,37,Goa Beaches,Goa,Beach,8.874758,Nov-Mar
2,884,Kerala Backwaters,Kerala,Nature,9.241445,Sep-Mar
3,737,Goa Beaches,Goa,Beach,9.413701,Nov-Mar
4,882,Goa Beaches,Goa,Beach,8.449238,Nov-Mar


## Testing

In [13]:
def split_user_histories(userhistory_df, test_size=0.2):
    # Inicializamos listas vacías para almacenar los subconjuntos de entrenamiento y prueba por usuario
    train_list = []
    test_list = []

    # Agrupamos el historial por usuario
    for user_id, group in userhistory_df.groupby("UserID"):
        # Si el usuario tiene menos de 2 interacciones, lo excluimos de la partición (no se puede dividir)
        if len(group) < 2:
            continue 
        
        # Dividimos aleatoriamente las interacciones del usuario en entrenamiento y prueba
        train, test = sklearn_train_test_split(group, test_size=test_size, random_state=42)
        
        # Agregamos los resultados a las listas correspondientes
        train_list.append(train)
        test_list.append(test)

    # Concatenamos todos los subconjuntos individuales en DataFrames finales
    return pd.concat(train_list), pd.concat(test_list)

# Aplicamos la función para dividir el historial original
train_history, test_history = split_user_histories(userhistory_df)


In [14]:
def evaluate_precision_at_k_cbf(userhistory_df, destinations_df, cosine_sim, test_history, k=5):
    # Lista para almacenar la precisión de cada usuario
    precisions = []
    
    # Obtenemos la lista única de usuarios
    user_ids = userhistory_df['UserID'].unique()

    for user_id in user_ids:
        # Obtenemos los destinos reales del conjunto de prueba para este usuario
        true_destinations = set(test_history[test_history['UserID'] == user_id]['DestinationID'])

        # Si no hay destinos en test para el usuario, lo saltamos
        if len(true_destinations) == 0:
            continue

        try:
            # Obtenemos las recomendaciones basadas en contenido para el usuario
            recommendations = recommend_destinations(
                user_id=user_id,
                userhistory_df=userhistory_df,
                destinations_df=destinations_df,
                cosine_sim=cosine_sim
            )
        except IndexError:
            # Si ocurre un error (por ejemplo, usuario no encontrado), continuamos con el siguiente usuario
            continue

        # Tomamos los IDs de destinos recomendados (hasta top k)
        recommended_ids = set(recommendations['DestinationID'].values[:k])
        
        # Calculamos el número de hits (recomendaciones correctas)
        hits = recommended_ids & true_destinations

        # Calculamos precisión para el usuario actual
        precision = len(hits) / k
        precisions.append(precision)

    # Calculamos la precisión promedio en todos los usuarios evaluados
    avg_precision = np.mean(precisions) if precisions else 0.0
    print(f"Average Precision@{k}: {avg_precision:.4f}")
    return avg_precision


In [16]:
# Evaluamos la precisión@5 del recomendador basado en contenido
avg_precision = evaluate_precision_at_k_cbf(
    userhistory_df=train_history,     # Historial de usuario para entrenamiento
    destinations_df=destinations_df,  # Información de destinos
    cosine_sim=cosine_sim,            # Matriz de similitud coseno entre destinos
    test_history=test_history,        # Historial para evaluación (prueba)
    k=5                              # Número de recomendaciones a evaluar
)


Average Precision@5: 0.0000
